- Gensim models were done using gensim with lemmatization and stopwords as that was found to be the best method
- This file also trains a model using Gensim FastText 
- This file also contains the training for the Hugging Face BERT byte pair encoding model

# Training Gensim model on neuroscience papers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install python-docx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 60.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184505 sha256=7f2b58938271ed2271877f3f88a467f2b3dcee3583a735665d49b590ee3ac278
  Stored in directory: /root/.cache/pip/wheels/32/b8/b2/c4c2b95765e615fe139b0b17b5ea7c0e1b6519b0a9ec8fb34d
Successfully built python-docx


In [ ]:
!pip install glove_python-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.9/974.9 KB 34.9 MB/s eta 0:00:00


In [ ]:
from docx import Document
import nltk
nltk.download('punkt')
import re
from nltk import sent_tokenize
import pandas as pd
from nltk.corpus import stopwords
nltk.download('stopwords')
import pickle
import numpy as np
import glob

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from nltk.stem import WordNetLemmatizer
import nltk 
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
!git clone 'https://github.com/igorbrigadir/stopwords.git'

Cloning into 'stopwords'...
remote: Enumerating objects: 149, done.
remote: Total 149 (delta 0), reused 0 (delta 0), pack-reused 149
Receiving objects: 100% (149/149), 85.27 KiB | 14.21 MiB/s, done.
Resolving deltas: 100% (52/52), done.


In [ ]:
alir3z4_data = '/content/stopwords/en/alir3z4.txt'

more_stops = pd.read_csv('/content/stopwords/en/alir3z4.txt')
new_stops = list(more_stops["'ll"])

In [ ]:
DOMAIN_STOPS = {'pubmed', 'et', 'al', 'page'}
STOPWORDS =  set(stopwords.words('english') + stopwords.words('german') +  stopwords.words('dutch') + stopwords.words('french') +  stopwords.words('spanish')  + new_stops) | DOMAIN_STOPS
STOPWORDS = set(STOPWORDS)

In [ ]:
len(STOPWORDS)

2011

In [ ]:
'a' in STOPWORDS

True

In [ ]:
ROOT = "/content/drive/MyDrive/regen_x"

In [ ]:
def get_docx(file_path):
    doc = []
    for para in Document(file_path).paragraphs:
        if para.text == "":
            continue
        doc += (sent_tokenize(para.text.lower())) # we lower text here
    return doc


def get_start_stop():
    domain_stops = {'pubmed', 'et', 'al', 'page'}
    with open('/content/stopwords/en/alir3z4.txt', 'r') as fn:
        new_stops = [line.strip() for line in fn.readlines()]
    STOPWORDS =  set(stopwords.words('english') + stopwords.words('german') +  stopwords.words('dutch') + stopwords.words('french') +  stopwords.words('spanish')  + new_stops) | domain_stops

    fn = glob.glob(ROOT + '/data/start-words/*')
    ALL_STARTS = [pickle.load(open(f , 'rb')) for f in fn]
    STARTWORDS = {}
    for f in ALL_STARTS:
      STARTWORDS.update(f)
    STARTWORDS = set(STARTWORDS.keys())

    assert(type(STOPWORDS)==set and type(STARTWORDS)==set)
    return (STARTWORDS, STOPWORDS)

In [ ]:
STARTWORDS, STOPWORDS = get_start_stop()

In [ ]:
# from gensim.models import Word2Vec

# Optimizing Training Function

You **don't** want to do incremental training for the reasons given in [this answer](https://stackoverflow.com/questions/42746007/incremental-word2vec-model-training-in-gensim)

# Lemmatizing, Concatenating, and Training Gensim Model

In [ ]:
# Don't forget to mount drive

In [ ]:
from natsort import natsorted
import os

In [ ]:
ROOT = "/content/drive/MyDrive/regen_x"
NUM_BINS = 16

In [ ]:
# for lemmatization 
import spacy
# Initialize spacy 'en' model, keeping only tagger component needed for lemmatization
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [ ]:
def do_stemming(filtered):
	stemmed = []
	for f in filtered:
		stemmed.append(PorterStemmer().stem(f))
		#stemmed.append(LancasterStemmer().stem(f))
		#stemmed.append(SnowballStemmer('english').stem(f))
	return stemmed

def do_lemmatizing(filtered):
  # convert list to string 
  spacy_parsed_text = nlp(" ".join(filtered)) 
  # Get the lemma for each token in the parsed text 
  
  # I wanted to keep pronouns so not taking lemma if it's a pronoun but if you want to remove pronouns use below commented line 
  # return " ".join([token.lemma_ for token in doc])

  # return as list of words again 
  return [token.lemma_ if token.lemma_ != '-PRON-' else token.lower_ for token in spacy_parsed_text]

In [ ]:
# Uses lemmatization and stopwords 
def get_proc_docs_total(all_papers_path, STARTWORDS, STOPWORDS, max_papers=None, verbose=True):
  file_paths = all_papers_path

  print("Number of files: {}".format(len(file_paths)))
  if len(file_paths) == 0:
    # raise Exception("Folder has no files - maybe drive was not mounted?")
    pass 
  ## -- Collecting Papers from Given Year -- ##
  proc_docs = [] 

  counter = 1
  length = len(file_paths)
  for f in file_paths:
    doc = get_docx(f)
    
    for sentence in doc:
      # don't think we need to remove stopwords and such if we're training embeddings 
      # do lemmatization here as well 

      proc_sentence = [] 
      proc_sentence = [word for word in re.findall(r'\w+', sentence) if ((len(word) > 2) and (word not in STOPWORDS))]

      proc_sentence = do_lemmatizing(proc_sentence) 
      
      proc_docs.append(proc_sentence)  

    if(verbose):
      print("\t{}/{}".format(counter, length))
    counter += 1

    if max_papers != None:
      if counter == max_papers+1:
        break 

  return proc_docs

In [ ]:
# MODEL_PATH = "/content/drive/MyDrive/Colab Notebooks/NLP - Lab/WordEmbeddings/Models/Gensim_Lemmatized_All_Docs/"

# if not os.path.exists(MODEL_PATH):
#     print("New path created")
#     os.makedirs(MODEL_PATH)

In [ ]:
all_papers_path = glob.glob(ROOT + '/data/ocr_paper_COMPREHENSIVE/*/*.docx')
all_papers_path[160:170]

In [ ]:
### Train and Save Models for all time periods ##

# Have to make one giant list of all processed docs because unfortunately can't incrementally 
# train as described above 
counter = 0 

all_proc_docs = [] 

# Process the docs (stopwords and lemmatization)
all_proc_docs = get_proc_docs_total(all_papers_path, STARTWORDS, STOPWORDS, max_papers=None, verbose=True)

In [ ]:
# Train the embeddings! 
print("Training word embeddings for all papers...")
model = Word2Vec(sentences=all_proc_docs, min_count=1) 

# Sanity check 
print(model.wv.most_similar("eye", topn=10))

# Store just the words + their trained embeddings.
word_vectors = model.wv
word_vectors.save(MODEL_PATH + "{}.wordvectors".format("All Papers"))

# Manually create space 
# del all_proc_docs
# del word_vectors 
# del model 

In [ ]:
# Exceution of above function took: 1h 5min
# Processing docs: 1h 3min
# Training embeddings: 2 min 

# Gensim FastText

In [ ]:
from gensim.models import FastText

In [ ]:
all_papers_path = glob.glob('/content/drive/MyDrive/regen_x/data/ocr_paper_COMPREHENSIVE/*/*.docx')
all_papers_path[160:170]

['/content/drive/MyDrive/regen_x/data/ocr_paper_COMPREHENSIVE/1915/1915_R. H. Jocelyn Swan.docx',
 '/content/drive/MyDrive/regen_x/data/ocr_paper_COMPREHENSIVE/1915/1915_R. Ingebrigtsen.docx',
 '/content/drive/MyDrive/regen_x/data/ocr_paper_COMPREHENSIVE/1915/1915_The British Medical Journal.docx',
 '/content/drive/MyDrive/regen_x/data/ocr_paper_COMPREHENSIVE/1915/1915_T. N. Foulis.docx',
 '/content/drive/MyDrive/regen_x/data/ocr_paper_COMPREHENSIVE/1915/1915_W. Harris.docx',
 '/content/drive/MyDrive/regen_x/data/ocr_paper_COMPREHENSIVE/1916/1916_A. R. Robertson.docx',
 '/content/drive/MyDrive/regen_x/data/ocr_paper_COMPREHENSIVE/1916/1916_A.T. Mussen.docx',
 '/content/drive/MyDrive/regen_x/data/ocr_paper_COMPREHENSIVE/1916/1916_D. Orr.docx',
 '/content/drive/MyDrive/regen_x/data/ocr_paper_COMPREHENSIVE/1916/1916_Eijkman, C. .docx',
 '/content/drive/MyDrive/regen_x/data/ocr_paper_COMPREHENSIVE/1916/1916_J. COLLIER.docx']

In [ ]:
# Uses lemmatization and stopwords 
file_paths = all_papers_path

proc_docs = [] 

counter = 1
length = len(file_paths)
for f in file_paths:
  doc = get_docx(f)
  
  for sentence in doc:
    proc_sentence = [] 
    # changed to >=2 word length for molecules with smaller abbreviation 
    proc_sentence = [word for word in re.findall(r'\w+', sentence) if ((len(word) >= 2) and (word not in STOPWORDS))]

    # proc_sentence = do_lemmatizing(proc_sentence) 
    
    proc_docs.append(proc_sentence)  

  print("\t{}/{}".format(counter, length))
  counter += 1

	1/700
	2/700
	3/700
	4/700
	5/700
	6/700
	7/700
	8/700
	9/700
	10/700
	11/700
	12/700
	13/700
	14/700
	15/700
	16/700
	17/700
	18/700
	19/700
	20/700
	21/700
	22/700
	23/700
	24/700
	25/700
	26/700
	27/700
	28/700
	29/700
	30/700
	31/700
	32/700
	33/700
	34/700
	35/700
	36/700
	37/700
	38/700
	39/700
	40/700
	41/700
	42/700
	43/700
	44/700
	45/700
	46/700
	47/700
	48/700
	49/700
	50/700
	51/700
	52/700
	53/700
	54/700
	55/700
	56/700
	57/700
	58/700
	59/700
	60/700
	61/700
	62/700
	63/700
	64/700
	65/700
	66/700
	67/700
	68/700
	69/700
	70/700
	71/700
	72/700
	73/700
	74/700
	75/700
	76/700
	77/700
	78/700
	79/700
	80/700
	81/700
	82/700
	83/700
	84/700
	85/700
	86/700
	87/700
	88/700
	89/700
	90/700
	91/700
	92/700
	93/700
	94/700
	95/700
	96/700
	97/700
	98/700
	99/700
	100/700
	101/700
	102/700
	103/700
	104/700
	105/700
	106/700
	107/700
	108/700
	109/700
	110/700
	111/700
	112/700
	113/700
	114/700
	115/700
	116/700
	117/700
	118/700
	119/700
	120/700
	121/700
	122/700
	123/700
	

In [ ]:
# Train the embeddings!
from gensim.models.fasttext import FastText
print("Training fasttext word embeddings for all papers...")
model = FastText(sentences=proc_docs, min_count = 1 )

Training fasttext word embeddings for all papers...


In [ ]:
# Sanity check 
print(model.wv.most_similar("akt-3", topn=10))

# Store just the words + their trained embeddings.
word_vectors = model.wv
word_vectors.save("/content/drive/MyDrive/Colab Notebooks/NLP - Lab/WordEmbeddings/Models/Gensim_Lemmatized_All_Docs/FastText.wordvectors")

[('akt', 0.8763183951377869), ('akt1', 0.8671360015869141), ('rhoe', 0.8668363094329834), ('rhob', 0.8653501272201538), ('smad2', 0.8633884191513062), ('rho1', 0.8628695607185364), ('wjjak', 0.8625777363777161), ('rhomhanyi', 0.8620971441268921), ('rhogdi', 0.8587667942047119), ('smad1', 0.8587009310722351)]


# Byte Pair Encoding using Hugging Face Tokenizers

In [ ]:
all_papers_path = glob.glob('/content/drive/MyDrive/regen_x/data/ocr_paper_COMPREHENSIVE/*/*.docx')
all_papers_path[160:170]

In [ ]:
# DON"T lemmatize or use stopwords for Byte Pair Encoding  
def sentence_generator():
  file_paths = all_papers_path
  counter = 1
  length = len(file_paths)
  for f in file_paths:
    print("\t{}/{}".format(counter, length))
    doc = get_docx(f)
    yield doc # speed up by returning batch of sentences 

    counter += 1

In [ ]:
!pip install tokenizers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 96.2 MB/s eta 0:00:00


Bert WordPiece-style Encoding

In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece
bert_tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))

In [ ]:
from tokenizers import normalizers
from tokenizers.normalizers import NFD, Lowercase, StripAccents
bert_tokenizer.normalizer = normalizers.Sequence([NFD(), Lowercase(), StripAccents()])

In [ ]:
from tokenizers.pre_tokenizers import Whitespace
bert_tokenizer.pre_tokenizer = Whitespace()

In [ ]:
from tokenizers.trainers import WordPieceTrainer
trainer = WordPieceTrainer(vocab_size=30522, special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])

Byte-Pair Encoding

In [ ]:
# from tokenizers import Tokenizer
# from tokenizers.models import BPE
# tokenizer = Tokenizer(BPE(unk_token="[UNK]"))

In [ ]:
# from tokenizers.trainers import BpeTrainer
# trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])

In [ ]:
# from tokenizers.pre_tokenizers import ByteLevel
# tokenizer.pre_tokenizer = ByteLevel()

In [ ]:
# from tokenizers import normalizers
# from tokenizers.normalizers import NFD, StripAccents
# normalizer = normalizers.Sequence([NFD(), StripAccents()])
# tokenizer.normalizer = normalizer

In [ ]:
bert_tokenizer.train_from_iterator(sentence_generator(), trainer)
bert_tokenizer.save("/content/drive/MyDrive/Colab Notebooks/NLP - Lab/WordEmbeddings/Models/BPE/WordPiece_Normalized_ByteLevel_30522_words.json")

	1/700
	2/700
	3/700
	4/700
	5/700
	6/700
	7/700
	8/700
	9/700
	10/700
	11/700
	12/700
	13/700
	14/700
	15/700
	16/700
	17/700
	18/700
	19/700
	20/700
	21/700
	22/700
	23/700
	24/700
	25/700
	26/700
	27/700
	28/700
	29/700
	30/700
	31/700
	32/700
	33/700
	34/700
	35/700
	36/700
	37/700
	38/700
	39/700
	40/700
	41/700
	42/700
	43/700
	44/700
	45/700
	46/700
	47/700
	48/700
	49/700
	50/700
	51/700
	52/700
	53/700
	54/700
	55/700
	56/700
	57/700
	58/700
	59/700
	60/700
	61/700
	62/700
	63/700
	64/700
	65/700
	66/700
	67/700
	68/700
	69/700
	70/700
	71/700
	72/700
	73/700
	74/700
	75/700
	76/700
	77/700
	78/700
	79/700
	80/700
	81/700
	82/700
	83/700
	84/700
	85/700
	86/700
	87/700
	88/700
	89/700
	90/700
	91/700
	92/700
	93/700
	94/700
	95/700
	96/700
	97/700
	98/700
	99/700
	100/700
	101/700
	102/700
	103/700
	104/700
	105/700
	106/700
	107/700
	108/700
	109/700
	110/700
	111/700
	112/700
	113/700
	114/700
	115/700
	116/700
	117/700
	118/700
	119/700
	120/700
	121/700
	122/700
	123/700
	

In [ ]:
# Hard to say because its supposed to be encoding at byte level but maybe still splits by whitespace? 
output = bert_tokenizer.encode("Optic nerve regeneration is increased with akt-3. This is interesting")
print(output.ids)
bert_tokenizer.decode(output.ids)
print(output.tokens)

[693, 458, 668, 374, 1492, 395, 10197, 17, 23, 18, 475, 374, 2851]
['Ġoptic', 'Ġnerve', 'Ġregeneration', 'Ġis', 'Ġincreased', 'Ġwith', 'Ġakt', '-', '3', '.', 'Ġthis', 'Ġis', 'Ġinteresting']


In [ ]:
# Hard to say because its supposed to be encoding at byte level but maybe still splits by whitespace? 
output = bert_tokenizer.encode("o")
print(output.ids)
bert_tokenizer.decode(output.ids)
print(output.tokens)

[312, 318]
['Ġo', 'Ġp']
